In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/download.tensorflow.org/data/certificate/satellitehurricaneimages.zip \
    -O /tmp/satellitehurricaneimages.zip

/bin/bash: /home/danieldu/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
wget: /home/danieldu/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-10-12 22:02:58--  https://storage.googleapis.com/download.tensorflow.org/data/certificate/satellitehurricaneimages.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.160.123, 142.251.43.27, 142.251.42.251, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.160.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35360890 (34M) [application/zip]
Saving to: ‘/tmp/satellitehurricaneimages.zip’

/tmp/satellitehurri 100%[===================>]  33.72M  15.7MB/s    in 2.2s    

2023-10-12 22:03:00 (15.7 MB/s) - ‘/tmp/satellitehurricaneimages.zip’ saved [35360890/35360890]



In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
from tensorflow.keras.layers import InputLayer

local_zip = '/tmp/satellitehurricaneimages.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/satellitehurricaneimages'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

IMG_SIZE = (128, 128)
INP_SHAPE = (*IMG_SIZE, 3)
BATCH_SIZE = 64

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True)


test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=20,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=IMG_SIZE,
        batch_size=20,
        class_mode='binary')

Found 10000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers.experimental.preprocessing import Resizing


model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

2023-10-12 22:03:03.040942: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-12 22:03:03.066088: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-12 22:03:03.066321: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-12 22:03:03.067066: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 32)        4640      
                                                                 
 dropout (Dropout)           (None, 61, 61, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 64)        1

2023-10-12 22:03:03.067376: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-12 22:03:03.067542: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-12 22:03:03.067690: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-12 22:03:03.372400: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-12 22:03:03.372590: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

In [5]:

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])

history = model.fit(
      train_generator,
      epochs=50,
      validation_data=validation_generator,
      batch_size=256)

Epoch 1/50
500/500 [==============================] - 25s 49ms/step - loss: 0.1568 - acc: 0.9406 - val_loss: 0.1927 - val_acc: 0.9325
Epoch 2/50
500/500 [==============================] - 24s 49ms/step - loss: 0.1509 - acc: 0.9417 - val_loss: 0.1945 - val_acc: 0.9400
Epoch 3/50
500/500 [==============================] - 24s 49ms/step - loss: 0.1398 - acc: 0.9464 - val_loss: 0.1163 - val_acc: 0.9600
Epoch 4/50
500/500 [==============================] - 24s 49ms/step - loss: 0.1414 - acc: 0.9464 - val_loss: 0.1309 - val_acc: 0.9510
Epoch 5/50
500/500 [==============================] - 25s 49ms/step - loss: 0.1489 - acc: 0.9454 - val_loss: 0.1590 - val_acc: 0.9440
Epoch 6/50
500/500 [==============================] - 24s 48ms/step - loss: 0.1413 - acc: 0.9466 - val_loss: 0.1349 - val_acc: 0.9545
Epoch 7/50
500/500 [==============================] - 24s 49ms/step - loss: 0.1469 - acc: 0.9448 - val_loss: 0.1638 - val_acc: 0.9435
Epoch 8/50
500/500 [==============================] - 24s 49ms

In [6]:
model.save("mymodel.h5")